# Wan2GP A100 Native WebUI (Run-All)

This notebook is for **A100-style high-VRAM use** with native/full-precision behavior:
- no runtime auto-quantization
- no compile shortcuts
- no step-skipping cache shortcuts
- full-precision BF16 model path for Wan2.2 I2V A14B

Run all cells top-to-bottom. The last cell launches Wan2GP and exposes it through ngrok.

## 1) Verify GPU runtime

- Kaggle: `Settings -> Accelerator -> GPU`
- Colab: `Runtime -> Change runtime type -> GPU`

In [ ]:
import os
import subprocess

try:
    out = subprocess.check_output(
        ['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'],
        text=True,
    )
    print(out)
    if 'A100' not in out:
        print('WARNING: GPU is not A100. Notebook still works, but native/full-precision may OOM on smaller GPUs.')
except Exception as exc:
    hint = (
        'Kaggle: enable GPU in Settings -> Accelerator.'
        if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ
        else 'Colab: enable GPU in Runtime -> Change runtime type.'
    )
    raise RuntimeError(f'GPU not detected. {hint}') from exc

## 2) Paths and runtime config

In [ ]:
import os
from pathlib import Path

IS_KAGGLE = 'KAGGLE_KERNEL_RUN_TYPE' in os.environ or Path('/kaggle').exists()
WAN2GP_ROOT = Path('/kaggle/working/Wan2GP' if IS_KAGGLE else '/content/Wan2GP').resolve()

if IS_KAGGLE:
    tmp_root = Path('/kaggle/tmp') if Path('/kaggle/tmp').exists() else Path('/tmp')
    CACHE_ROOT = tmp_root / 'wan2gp-cache'
    MODEL_CACHE = CACHE_ROOT / 'models'
    CKPTS_CACHE = CACHE_ROOT / 'ckpts'
else:
    MODEL_CACHE = WAN2GP_ROOT / 'models'
    CKPTS_CACHE = WAN2GP_ROOT / 'ckpts'

print('Runtime:', 'Kaggle' if IS_KAGGLE else 'Colab')
print('WAN2GP_ROOT:', WAN2GP_ROOT)
print('MODEL_CACHE:', MODEL_CACHE)
print('CKPTS_CACHE:', CKPTS_CACHE)

## 3) Clone/update repo

In [ ]:
import subprocess

# Point this to the branch/repo that contains your native-mode changes.
REPO_URL = 'https://github.com/kanaya23/Wan2GP.git'

if WAN2GP_ROOT.exists():
    print('Repo exists, pulling latest...')
    subprocess.run(['git', '-C', str(WAN2GP_ROOT), 'pull'], check=True)
else:
    subprocess.run(['git', 'clone', REPO_URL, str(WAN2GP_ROOT)], check=True)

## 4) Install system + Python dependencies

In [ ]:
import os
import subprocess
import sys

env = os.environ.copy()
env.setdefault('DEBIAN_FRONTEND', 'noninteractive')

apt_prefix = ['apt-get'] if IS_KAGGLE else ['sudo', 'apt-get']
subprocess.run(apt_prefix + ['update', '-qq'], check=True, env=env)
subprocess.run(
    apt_prefix + [
        'install', '-y', '--no-install-recommends',
        'ffmpeg', 'libglib2.0-0', 'libgl1', 'libportaudio2'
    ],
    check=True,
    env=env,
)

subprocess.run([sys.executable, '-m', 'pip', 'install', '--upgrade', 'pip', 'setuptools', 'wheel'], check=True, env=env)
subprocess.run([
    sys.executable, '-m', 'pip', 'install',
    'torch==2.8.0', 'torchvision', 'torchaudio',
    '--index-url', 'https://download.pytorch.org/whl/cu128'
], check=True, env=env)
subprocess.run([
    sys.executable, '-m', 'pip', 'install',
    'xformers==0.0.32.post2',
    '--index-url', 'https://download.pytorch.org/whl/cu128'
], check=True, env=env)
subprocess.run([sys.executable, '-m', 'pip', 'install', '-r', str(WAN2GP_ROOT / 'requirements.txt')], check=True, env=env)
subprocess.run([sys.executable, '-m', 'pip', 'install', 'hydra-core', 'omegaconf', 'pyngrok'], check=True, env=env)

print('Dependencies installed.')

## 5) Enforce full-precision model override + native-mode guards

In [ ]:
    import json
    from pathlib import Path

    finetunes_dir = WAN2GP_ROOT / 'finetunes'
    finetunes_dir.mkdir(parents=True, exist_ok=True)

    i2v_override = {
        'model': {
            'URLs': [
                'https://huggingface.co/DeepBeepMeep/Wan2.2/resolve/main/wan2.2_image2video_14B_high_mbf16.safetensors'
            ],
            'URLs2': [
                'https://huggingface.co/DeepBeepMeep/Wan2.2/resolve/main/wan2.2_image2video_14B_low_mbf16.safetensors'
            ],
            'text_encoder_folder': 'umt5-xxl',
            'text_encoder_URLs': [
                'https://huggingface.co/DeepBeepMeep/Wan2.1/resolve/main/umt5-xxl/models_t5_umt5-xxl-enc-bf16.safetensors'
            ],
            'auto_quantize': False,
        }
    }

    override_file = finetunes_dir / 'i2v_2_2.json'
    override_file.write_text(json.dumps(i2v_override, indent=4))
    print('Wrote', override_file)

    wgp_text = (WAN2GP_ROOT / 'wgp.py').read_text(encoding='utf-8')
    required = [
        'attention_mode = "sdpa"',
        'compile = ""',
        'skip_steps_cache_type = ""',
        'quantizeTransformer = False',
        'def compute_profile(override_profile, output_type="video"):
    return 1',
    ]
    missing = [s for s in required if s not in wgp_text]
    if missing:
        raise RuntimeError(
            'Native-mode code guards missing in this repo checkout. Use your patched branch. Missing snippets:
' + '
'.join(missing)
        )

    print('Native-mode guard check passed.')

## 6) Launch WebUI on ngrok (run and keep this cell alive)

In [ ]:
import os
import shutil
import subprocess
import sys
import threading
import time

from pyngrok import ngrok

env = os.environ.copy()

token = env.get('NGROK_AUTHTOKEN', '')
if not token and IS_KAGGLE:
    try:
        from kaggle_secrets import UserSecretsClient
        token = UserSecretsClient().get_secret('NGROK_AUTHTOKEN')
        if token:
            env['NGROK_AUTHTOKEN'] = token
    except Exception:
        pass

if not token:
    raise RuntimeError('Missing NGROK_AUTHTOKEN. Set it in environment or Kaggle secrets.')

MODEL_CACHE.mkdir(parents=True, exist_ok=True)
CKPTS_CACHE.mkdir(parents=True, exist_ok=True)

def link_dir(src_dir, cache_dir):
    if src_dir.is_symlink():
        if src_dir.resolve() != cache_dir:
            src_dir.unlink()
            src_dir.symlink_to(cache_dir, target_is_directory=True)
        return
    if src_dir.exists():
        if src_dir.is_dir():
            for child in src_dir.iterdir():
                dst = cache_dir / child.name
                if dst.exists():
                    if child.is_dir():
                        shutil.rmtree(child)
                    else:
                        child.unlink()
                else:
                    shutil.move(str(child), str(dst))
            src_dir.rmdir()
        else:
            src_dir.unlink()
    src_dir.symlink_to(cache_dir, target_is_directory=True)

if IS_KAGGLE:
    link_dir(WAN2GP_ROOT / 'models', MODEL_CACHE)
    link_dir(WAN2GP_ROOT / 'ckpts', CKPTS_CACHE)

env['WAN_CACHE_DIR'] = str(MODEL_CACHE)
if IS_KAGGLE:
    env['PYTHONPATH'] = f"{WAN2GP_ROOT}:{env.get('PYTHONPATH', '')}"
    env.setdefault('HF_HOME', str(MODEL_CACHE / 'hf_home'))
    env.setdefault('HUGGINGFACE_HUB_CACHE', str(MODEL_CACHE / 'hf_home' / 'hub'))
    env.setdefault('TRANSFORMERS_CACHE', str(MODEL_CACHE / 'hf_home' / 'transformers'))

ngrok.set_auth_token(token)
tunnel = ngrok.connect(addr='7860', bind_tls=True)
print('Ngrok URL:', tunnel.public_url)

cmd = [
    sys.executable,
    '-u',
    'wgp.py',
    '--listen',
    '--server-port', '7860',
    '--bf16',
]

print('Launching Wan2GP:', ' '.join(cmd))
process = subprocess.Popen(
    cmd,
    cwd=str(WAN2GP_ROOT),
    env=env,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,
)

stop_event = threading.Event()

def keepalive_loop():
    while not stop_event.is_set():
        time.sleep(45)
        if stop_event.is_set():
            break
        print('[keepalive] Wan2GP is still running...')

thread = threading.Thread(target=keepalive_loop, daemon=True)
thread.start()

try:
    for line in iter(process.stdout.readline, ''):
        if not line:
            break
        print(line, end='')
except KeyboardInterrupt:
    print('Stopping Wan2GP...')
    process.terminate()
finally:
    stop_event.set()
    try:
        process.wait(timeout=15)
    except Exception:
        pass
    thread.join(timeout=1)
    try:
        ngrok.disconnect(tunnel.public_url)
    except Exception:
        pass
    print(f'Wan2GP stopped (return code: {process.returncode}).')